# Evaluation Results of Rare Adverse Drug Reactions (ADRs) in SIDER

Import necessary modules: 

In [1]:
import numpy as np
import pandas as pd
from ADRprofilePrediction import Pairs2Mat, evaluation
from Models import loadHyperpar
import json

In [2]:
import sklearn
print(sklearn.__version__)

0.24.2


## Load data

Load the feature data in to a dictionary. Drug-target, drug-enzyme, drug-chemical structure fingerprint, drug-gene interaction, drug-transporter, drug-pathway and drug-indication are included.

In [3]:
features_dict = {
    "target":Pairs2Mat(path="data/drug_target.tsv",colname1="0",colname2="1"),
    "enzyme":Pairs2Mat(path="data/drug_enzyme.tsv",colname1="0",colname2="1"),
    "Chem":pd.read_csv("data/drug_chemsfp.tsv",sep = "\t",header=0,index_col=0),
    "DGI":Pairs2Mat(path="data/interactions.tsv",colname1="drug_claim_name",colname2="gene_name"),
    "transporter":Pairs2Mat(path="data/drug_transporter.tsv",colname1="0",colname2="1"),
    "pathway":Pairs2Mat(path="data/drug_pathway.tsv",colname1="0",colname2="1"),
    "indication":Pairs2Mat(path="data/drug_indication.tsv",colname1="1_x",colname2="6")
}


Load ADR data from SIDER and OFFSIDES. Variable SEs is a dict that stores ADR data. Variable filter controls the frequency of the ADR. When filter is "all", only the ADRs with extremely low frequencies are removed; when filter is "rare" only frequency less than 50 were used.


In [4]:
filter = "rare"
SEs = {}
if filter == "all":
    SIDER = Pairs2Mat(path="data/drug_se.tsv",colname1="1_x",colname2="5")
    column_sums = np.sum(SIDER, axis=0)
    SEs["SIDER"] = SIDER.loc[:, (column_sums >= 5)]

    OFFSIDERS = Pairs2Mat(path="data/OFFSIDES.csv",colname1="drug_concept_name",colname2="condition_concept_name",sep = ",")
    column_sums = np.sum(OFFSIDERS, axis=0)
    SEs["OFFSIDES"] = OFFSIDERS.loc[:, column_sums >= 5]
elif filter == "rare":
    SIDER = Pairs2Mat(path="data/drug_se.tsv",colname1="1_x",colname2="5")
    column_sums = np.sum(SIDER, axis=0)
    SEs["SIDER"] = SIDER.loc[:, (column_sums < 50)]

    OFFSIDERS = Pairs2Mat(path="data/OFFSIDES.csv",colname1="drug_concept_name",colname2="condition_concept_name",sep = ",")
    column_sums = np.sum(OFFSIDERS, axis=0)
    SEs["OFFSIDES"] = OFFSIDERS.loc[:, column_sums < 50]



## Set variables

The variables below includes all the options for the code.

- features_names: This varible is the list of all the features including the target feature, the enzyme feature, chemical structure fingerprint (Chem), drug-gene interaction (DGI), the transporter featrue, the pathway feature, the indication feature.
- SE_names: ADR data from SIDER.
- methods: This option is machine learning methods used for prediction, including Smoothed Kernel Regression (SKR), Kernel Regression (KR), the na\"ive method (Naive), Linear Neighbourhood Similarity Method using Regularized Linear Neighbour Similarity or Jaccard similarity (LNSM_RLN, LNSM_jaccard), Support Vector Machine (SVM), Random Forest (RF) and Boosted Random Forest (BRF).
- metrice_names: Metric we used to evaluate the performance of methods: AUPR, AUROC, AUPR per drug, AUROC per drug, AUPR+AUROC and AUPR+AUROC per drug.
- SE_name: The used ADR data in this file.
- metric: We used AUPR as the tuning metrice in Nested CV and CV.

In [7]:
features_names = ["target", "enzyme", "Chem", "DGI", "transporter", "pathway", "indication"]
# SEs_names = ["SIDER", "OFFSIDES"]
# methods = ["SKR", "KR", "KRR", "Naive", "LNSM_RLN", "LNSM_jaccard", "VKR"]
methods = ["SKR", "KRR", "VKR", "Naive", "LNSM_RLN", "LNSM_jaccard"]
# methods = ["SKR", "KR", "KRR", "Naive", "LNSM_RLN", "LNSM_jaccard", "VKR", "SVM", "OCCA", "SCCA", "RF", "BRF"]
tuning_metrices=["AUROC", "AUPR", "AUROCperdrug", "AUPRperdrug"]
metrice_names = ["AUPR+AUROC", "AUPR+AUROCperdrug", "AUROC", "AUPR", "AUROCperdrug", "AUPRperdrug"]
SEs_name = "SIDER"
metrice = "AUPR"

Set the variables for hyperparameters. We summarized 3 types of hyperparameters (SVM, RF and BRF are not competitive and time-consuming, and were tuned and trained in a seperated file -- SVM_RF.ipynb): 
 - A: This hyperparameters are tuned according to the step $\dots, 10^{-1}, 10^{0}, 10^{1}, \dots$ ($\sigma_X$ and $\sigma_Y$ does not change during tuning so they can be set as $10$ and $100$ respectively).
 - B: This hyperparameters are in $[0,1]$ and tuned according to the step $0, 0.1, \dots, 1$.
 - C: This hyperparameters are tuned based on $5, 10, 15, \dots$.

In [8]:
A = 10**np.arange(-2, 3, 1, dtype=float)
B = np.arange(0.1, 1, 0.1, dtype=float)
C = np.arange(5, 20, 5, dtype=int)
A10 = 10**np.arange(1, 2, 1, dtype=float)
A100 = 10**np.arange(2, 3, 1, dtype=float)
all_hyperparlist = {
    "SKR":[A,B,A10,A100], 
    # "KR":[A,A], 
    "KRR":[A,A],
    "VKR":[A,A,C], 
    "Naive":[], 
    "LNSM_RLN":[B,A], 
    "LNSM_jaccard":[B], 
    # "SVM":[A,A,A], 
    # "OCCA":[], 
    # "SCCA":[A], 
    # "RF":[C], 
    # "BRF":[C]
}

Set dictionaries to store the tuned hyperparameters and the results of CV and Nested CV.

In [10]:
hyperparsOut = {}
hyperparsOut["nested_cv"] = {}
hyperparsOut["cv"] = {}
results = {}
results["nested_cv"] = {}
results["cv"] = {}

## Nested CV and CV

Load tuned hyperparameters. If fully rerunning the tuning step of Nested CV and CV is required, please skip loading variable hyperpars and remove the option `hyperparfixed` of the function `evaluation()`.

In [17]:
# Open and read the JSON file
with open(f'results/hyperpars_{SEs_name}.xml', 'r') as xml_file:
    hyperpars = json.load(xml_file)

In [18]:
for method in methods:
    validation = "nested_cv"
    hyperparsOut[validation][method] = {}
    results[validation][method] = {}
    for str in features_names:
        print(f"using feature {str}")
        # Load the hyperparameter combination
        hyperparList = loadHyperpar(*all_hyperparlist[method],method_option=method)
        results[validation][method][str], hyperparsOut[validation][method][str] = evaluation(Y=SEs["SIDER"], X=features_dict[str], method_option=method,tuning_metrice=metrice,hyperparList=hyperparList,hyperparfixed=hyperpars[validation][method][str],Validation=validation,n_jobs=1)

    validation = "cv"
    hyperparsOut[validation][method] = {}
    results[validation][method] = {}
    for str in features_names:
        print(f"using feature {str}")
        hyperparList = loadHyperpar(*all_hyperparlist[method],method_option=method)
        results[validation][method][str], hyperparsOut[validation][method][str] = evaluation(Y=SEs["SIDER"], X=features_dict[str], method_option=method,tuning_metrice=metrice,hyperparList=hyperparList,hyperparfixed=hyperpars[validation][method][str],Validation=validation,n_jobs=1)

using feature target
The SKR requires hyperparameter lambda, c, sigma_X, sigma_Y
---------- nested cv start ----------
Fold: 0
number of hyperpars combination:  45
first few training idx:  [116 159 182 247 255 281 435 618 622 656]
first few testing idx:  [ 23 189 261 291 396 400 476 604 649 690]
--- tuning end ---
target size: 138
------ best hyper pars:  [0.01, 0.4, 10, 100] ------
SKR starts:
SKR ends:
-----------
AUPRperdrug: 0.09901261079785018
AUROCperdrug: 0.773763646126857
AUPR+AUROCperdrug: 0.8727762569247072
AUPR: 0.048708768719136736
AUROC: 0.7365719741328659
AUPR+AUROC: 0.7852807428520027
-----------
Fold: 1
number of hyperpars combination:  45
first few training idx:  [ 23 189 261 291 396 400 476 604 649 690]
first few testing idx:  [116 159 182 247 255 281 435 618 622 656]
--- tuning end ---
target size: 138
------ best hyper pars:  [0.01, 0.4, 10, 100] ------
SKR starts:
SKR ends:
-----------
AUPRperdrug: 0.12328826353183282
AUROCperdrug: 0.7770977355108829
AUPR+AUROCperd

## Save Output

Store the tuned hyperparameters for reproducing results.

In [ ]:
# with open(f'results/hyperpars_{SEs_name}.xml', 'w') as xml_file:
#    json.dump(hyperparsOut, xml_file)

Store the results of Nested CV and CV.

In [19]:
with open(f'results/results_{SEs_name}_{filter}.xml', 'w') as xml_file:
   json.dump(results, xml_file)

## Reorganize the Results and Calculate the P-value of Method Comparison

Load the results of Nested CV and CV.

In [20]:
with open(f'results/results_{SEs_name}_{filter}.xml', 'r') as xml_file:
    results = json.load(xml_file)

Orgainize the results of Nested CV into table.

In [21]:
df = pd.DataFrame()
for m, fs in results["nested_cv"].items():
    for f, mes in fs.items():
        for me, scores in mes.items():
            temp_df = pd.DataFrame({
                'method': m,
                'feature': f,
                'metric': me,
                "score": scores
            })
            df = pd.concat([df, temp_df], ignore_index=True)

custom_order = ["pathway","Chem", "DGI",  "indication", "target", "transporter", "enzyme"]
df['feature'] = pd.Categorical(df['feature'], categories=custom_order, ordered=True)
df['method'] = pd.Categorical(df['method'], categories=methods, ordered=True)
df['metric'] = pd.Categorical(df['metric'], categories=metrice_names, ordered=True)
df2 = pd.pivot_table(df, values=['score'], index=["feature", "method"], aggfunc={'score': ["mean","std"]}, columns=["metric"])
df3 = df2.sort_index(axis=1, level='metric').sort_index(level='feature')
df3.to_excel(f'results/nested_cv_results_{SEs_name}_{filter}.xlsx')
df3

score                               \
                               mean        std              mean   
metric                   AUPR+AUROC AUPR+AUROC AUPR+AUROCperdrug   
feature     method                                                 
pathway     SKR            0.835042   0.015121          0.916771   
            KRR            0.792077   0.021675          0.875733   
            VKR            0.799172   0.016793          0.804518   
            Naive          0.791896   0.011867          0.813665   
            LNSM_RLN       0.717886   0.030127          0.763659   
            LNSM_jaccard   0.605649   0.076976          0.657681   
Chem        SKR            0.799193   0.034896          0.857009   
            KRR            0.767846   0.035998          0.828240   
            VKR            0.770189   0.011737          0.785314   
            Naive          0.795526   0.006659          0.816770   
            LNSM_RLN       0.807919   0.036328          0.863521   
            LNSM_jaccard   0.512384   0.019665          0.521511   
DGI         SKR            0.783661   0.013205          0.849980   
            KRR            0.769053   0.013637          0.834155   
            VKR            0.773871   0.021402          0.772966   
            Naive          0.785489   0.005799          0.804576   
            LNSM_RLN       0.662569   0.024161          0.685786   
            LNSM_jaccard   0.519841   0.026244          0.531134   
indication  SKR            0.817447   0.021294          0.862328   
            KRR            0.810985   0.021653          0.852890   
            VKR            0.799529   0.031584          0.773931   
            Naive          0.785978   0.005667          0.803743   
            LNSM_RLN       0.708298   0.040158          0.698982   
            LNSM_jaccard   0.672312   0.031092          0.736525   
target      SKR            0.803289   0.019200          0.885943   
            KRR            0.803609   0.016149          0.886464   
            VKR            0.766297   0.017672          0.790806   
            Naive          0.791917   0.004910          0.813104   
            LNSM_RLN       0.640118   0.055510          0.652773   
            LNSM_jaccard   0.581575   0.057314          0.620568   
transporter SKR            0.774263   0.013933          0.804246   
            KRR            0.775350   0.011070          0.804929   
            VKR            0.760398   0.017762          0.761437   
            Naive          0.786424   0.007305          0.805034   
            LNSM_RLN       0.715230   0.023252          0.722180   
            LNSM_jaccard   0.451388   0.034855          0.434722   
enzyme      SKR            0.776731   0.024420          0.800188   
            KRR            0.766223   0.027136          0.791564   
            VKR            0.777781   0.014600          0.784162   
            Naive          0.795753   0.012765          0.814066   
            LNSM_RLN       0.724025   0.038577          0.726297   
            LNSM_jaccard   0.501821   0.010190          0.474950   

                                                                          \
                                       std      mean       std      mean   
metric                   AUPR+AUROCperdrug     AUROC     AUROC      AUPR   
feature     method                                                         
pathway     SKR                   0.024795  0.753108  0.008690  0.081934   
            KRR                   0.028389  0.711942  0.015692  0.080135   
            VKR                   0.008558  0.760231  0.008005  0.038941   
            Naive                 0.005101  0.767493  0.012370  0.024404   
            LNSM_RLN              0.036469  0.644088  0.022965  0.073798   
            LNSM_jaccard          0.060135  0.580941  0.074165  0.024709   
Chem        SKR                   0.025752  0.728132  0.024448  0.071061   
            KRR                   0.023691  0.697571  0.025012  0

Orgainize the results of CV into table.

In [22]:
df = pd.DataFrame()
for m, fs in results["cv"].items():
    for f, mes in fs.items():
        for me, scores in mes.items():
            temp_df = pd.DataFrame({
                'method': m,
                'feature': f,
                'metric': me,
                "score": scores
            },index=["1"])
            df = pd.concat([df, temp_df], ignore_index=True)
df['feature'] = pd.Categorical(df['feature'], categories=custom_order, ordered=True)
df['method'] = pd.Categorical(df['method'], categories=methods, ordered=True)
df['metric'] = pd.Categorical(df['metric'], categories=metrice_names, ordered=True)
df2 = pd.pivot_table(df, values=['score'], index=["feature", "method"], columns="metric")
df2.to_excel(f'results/cv_results_{SEs_name}_rare.xlsx')
df2

score                                        \
metric                   AUPR+AUROC AUPR+AUROCperdrug     AUROC      AUPR   
feature     method                                                          
pathway     SKR            0.800483          0.832402  0.771716  0.028767   
            KRR            0.802500          0.832841  0.772980  0.029520   
            VKR            0.783390          0.792441  0.756020  0.027371   
            Naive          0.796940          0.824556  0.768120  0.028821   
            LNSM_RLN       0.742350          0.784023  0.667441  0.074908   
            LNSM_jaccard   0.418612          0.494799  0.407967  0.010645   
Chem        SKR            0.834835          0.878129  0.782746  0.052089   
            KRR            0.808519          0.838999  0.775254  0.033265   
            VKR            0.799166          0.795581  0.756581  0.042585   
            Naive          0.791128          0.816394  0.765827  0.025301   
            LNSM_RLN       0.800070          0.865896  0.734317  0.065753   
            LNSM_jaccard   0.521018          0.533893  0.510114  0.010903   
DGI         SKR            0.820121          0.848577  0.779703  0.040418   
            KRR            0.796386          0.817449  0.770216  0.026170   
            VKR            0.775438          0.778167  0.751571  0.023867   
            Naive          0.790185          0.810818  0.765628  0.024557   
            LNSM_RLN       0.676160          0.703124  0.621638  0.054522   
            LNSM_jaccard   0.639180          0.683903  0.618631  0.020549   
indication  SKR            0.872658          0.884526  0.795474  0.077185   
            KRR            0.789998          0.817133  0.767830  0.022168   
            VKR            0.779900          0.778254  0.757142  0.022759   
            Naive          0.793659          0.814666  0.770672  0.022987   
            LNSM_RLN       0.786812          0.764752  0.684429  0.102383   
            LNSM_jaccard   0.710791          0.726498  0.649531  0.061260   
target      SKR            0.795248          0.818455  0.768428  0.026820   
            KRR            0.797217          0.818630  0.768766  0.028451   
            VKR            0.777324          0.781173  0.751416  0.025908   
            Naive          0.789075          0.809474  0.764326  0.024749   
            LNSM_RLN       0.657662          0.683875  0.601774  0.055888   
            LNSM_jaccard   0.542969          0.572484  0.522375  0.020594   
transporter SKR            0.809039          0.831097  0.777431  0.031608   
            KRR            0.807857          0.827640  0.776350  0.031507   
            VKR            0.787307          0.795286  0.758160  0.029147   
            Naive          0.805878          0.827300  0.774668  0.031210   
            LNSM_RLN       0.745858          0.740246  0.715408  0.030450   
            LNSM_jaccard   0.426207          0.416588  0.417373  0.008833   
enzyme      SKR            0.799238          0.819450  0.767902  0.031336   
            KRR            0.796314          0.814063  0.766354  0.029960   
            VKR            0.779501          0.781556  0.751041  0.028460   
            Naive          0.793378          0.810952  0.764411  0.028968   
            LNSM_RLN       0.752890          0.763860  0.719783  0.033108   
            LNSM_jaccard   0.488316          0.478071  0.479115  0.009200   

                                                   
metric                   AUROCperdrug AUPRperdrug  
feature     method                                 
pathway     SKR              0.795311    0.037091  
            KRR              0.795415    0.037426  
            VKR              0.759717    0.032724  
            Naive            0.789031    0.035525  
            LNSM_RLN         0.665453    0.118570  
            LNSM_jaccard     0.442719    0.052080  
Chem        SKR              0.805465    0.072664  
            KRR              0.795312    0.0

Calculate the P-value for method comparison based on the result of nested CV.

In [23]:
df = pd.DataFrame()
for m, fs in results["nested_cv"].items():
    for f, mes in fs.items():
        for me, scores in mes.items():
            temp_df = pd.DataFrame({
                'method': m,
                'feature': f,
                'metric': me,
            }, index=["1"])
            temp_df2 = pd.concat([temp_df, pd.DataFrame(scores, columns=["1"]).T], axis=1)
            df = pd.concat([df, temp_df2], ignore_index=True)
for m in metrice_names:
    for f in features_names:
        df2 = df[(df["metric"] == m) & (df["feature"] == f)]
        df3 = df2.iloc[:, np.array([0, 3, 4, 5, 6, 7])]
        df4 = df3.set_index(df3.columns[0])
        df5 = df4.T.ptests(paired=True, stars=False)
        df5.to_excel(f'results/pvalue_{SEs_name}_{filter}_{f}_{m}.xlsx')